# SOT

In [5]:
import cv2
import os

tracker = cv2.TrackerMIL_create()  # метод Multiple Instance Learning
tracker = cv2.legacy.TrackerMOSSE_create()  # Minimum Output Sum of Squared Error
tracker = cv2.TrackerGOTURN_create() # нейросеть (требует веса)

In [ ]:
#pip uninstall opencv-python  
#pip install opencv-contrib-python

При нажатии на m входим в режим выделения объекта и далее можем выйти из режима по нажатии на q

In [7]:
cap = cv2.VideoCapture(0)
while True:
    timer = cv2.getTickCount()
    success, img = cap.read()
    cv2.imshow("Tracking", img)
    if not success:
        break
    if cv2.waitKey(1) & 0xff == ord('m'):
        break 

bbox = cv2.selectROI("Tracking", img)
tracker.init(img, bbox)
  

def drawBox():
    cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), (255, 0, 0), 3, 1)

while True:
    timer = cv2.getTickCount()
    success, img = cap.read()
    success, bbox = tracker.update(img)
    if success:
        drawBox()
    else:
        cv2.putText(img, "Lost", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(img, str(int(fps)), (75, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Tracking", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
cap = cv2.VideoCapture('police.mp4')
while True:
    timer = cv2.getTickCount()
    success, img = cap.read()
    cv2.imshow("Tracking", img)
    if not success:
        break
    if cv2.waitKey(50) & 0xff == ord('m'):
        break  
    
bbox = cv2.selectROI("Tracking", img)
tracker.init(img, bbox)

def drawBox():
    cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), (255, 0, 0), 3, 1)

while True:
    timer = cv2.getTickCount()
    success, img = cap.read()
    if not success:
        break
    success, bbox = tracker.update(img)
    if success:
        drawBox()
    else:
        cv2.putText(img, "Lost", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(img, str(int(fps)), (75, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Tracking", img)
    if cv2.waitKey(10) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

---

## АНТИДИСТОРСИЯ:

In [8]:
import cv2
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display

def undistort_image(image, k1, k2, fx, fy):
    h, w = image.shape[:2]
    #fx = fy = 1.0  # фокусное расстояние, можно настроить
    cx, cy = w / 2, h / 2  # центр изображения

    # Матрица камеры
    camera_matrix = np.array([[fx, 0, cx],
                              [0, fy, cy],
                              [0, 0, 1]])

    # Коэффициенты дисторсии
    dist_coeffs = np.array([(k1), (k2), 0, 0, 0])

    # Используем функцию undistort из OpenCV
    undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs)

    return undistorted_image

# Создаем интерактивные виджеты для коэффициентов дисторсии
k1_slider = widgets.FloatSlider(value=0.0, min=-0.000001, max=0.000001, step=0.0000001, description='k1:')
k2_slider = widgets.FloatSlider(value=0.0, min=-0.0000000001, max=0.0000000001, step=0.00000000001, description='k2:')
fx_slider = widgets.FloatSlider(value=1.0, min=0, max=5, step=0.01, description='fx:')
fy_slider = widgets.FloatSlider(value=1.0, min=0, max=5, step=0.01, description='fy:')

# Создаем функцию-обработчик изменений слайдеров
def update_image(k1, k2, fx, fy):
    undistorted = undistort_image(original_image, k1, k2, fx, fy)
    undistorted = cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB) 
    plt.imshow(undistorted)

# Загружаем изображение
original_image = cv2.imread('test.jpg')

# Создаем интерактивные виджеты
interactive_widgets = widgets.interactive(update_image, k1=k1_slider, k2=k2_slider, fx=fx_slider, fy=fy_slider)

# Отображаем изображение и виджеты
display(interactive_widgets)


interactive(children=(FloatSlider(value=0.0, description='k1:', max=1e-06, min=-1e-06, step=1e-07), FloatSlide…